### Langgraph로 RAG 구축하기

#### 모델 선언하기

In [91]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemma3:4b", model_provider="ollama")

#### 문서에서 텍스트 추출 및 청킹

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_qdrant import qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader(
    file_path="data/arxiv_paper.pdf",
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(splits[0].page_content)

#### 임베딩 모델 로드

In [93]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="bge-m3")

#### Qdrant 벡터DB 저장 및 검색기 설정

In [94]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4

client = QdrantClient(path="/tmp/0416(1)/arxiv_paper")

# 밀집 벡터로 컬렉션 생성
client.create_collection(
    collection_name="embodied_agent_0415",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

qdrant = QdrantVectorStore(
    client=client,
    collection_name="embodied_agent_0415",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
)

uuids = [str(uuid4()) for _ in range(len(splits))]

qdrant.add_documents(documents=splits, ids=uuids)

# 벡터 저장소를 검색기로 설정
retriever = qdrant.as_retriever()

In [ ]:
search_result=retriever.invoke("Embodied Agent가 뭐야?")

for doc in search_result:
    print(doc.page_content[:500])
    print(doc.metadata)
    print("-"*100)

#### 프롬프트 템플릿 설정

In [96]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{context}

질문:
{question}

답변:
""")

#### Langgraph State, node, edge 선언하기

In [97]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

#### 답변 스트리밍하기

In [ ]:
for message, metadata in graph.stream(
    {"question": "Embodied Agent가 뭐야?"}, stream_mode="messages"
):
    print(message.content, end="")

#### Structured Output으로 답변에 주석 달기 - GPT-4.1

In [100]:
from typing import List

from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 Q&A 전문 AI 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답변해주세요.

컨텍스트:
{context}

질문:
{question}

답변:
""")

# Desired schema for response
class AnswerWithSources(TypedDict):
    """An answer to the question, with sources."""

    answer: str
    sources: Annotated[
        List[str],
        ...,
        "List of sources (title + page number + year ) used to answer the question",
    ]


class State(TypedDict):
    question: str
    context: List[Document]
    answer: AnswerWithSources


def generate(state: State):
    docs_content = "\n\n".join(str((doc.metadata,doc.page_content)) for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)
    structured_llm = llm.with_structured_output(AnswerWithSources)
    response = structured_llm.invoke(messages)
    return {"answer": response}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
import json

result = graph.invoke({"question": "Embodied Agent가 뭐야?"})
result

In [ ]:
print(result['answer']['sources'])

#### XML로 답변에 참고 문서 주석달기 - Gemma3:4b

In [105]:
from typing import List, Dict

from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import XMLOutputParser
from langgraph.checkpoint.memory import MemorySaver

# XML 시스템 프롬프트 정의
xml_system = """
You're a helpful AI assistant. 
Given a user question and some document snippets,answer the user question and provide citations.
If none of the documents answer the question, just say you don't know.

Remember,You should return the source information(title, page, year) and answer. 
A citation consists of a VERBATIM quote that justifies the answer and the source information. 
Return a citation for every quote across all documents that justifies the answer. 

Use the following format for your final output:
<cited_answer>
    <answer></answer>
    <citations>
        <citation><source></source><quote></quote></citation>
        <citation><source></source><quote></quote></citation>
        ...
    </citations>
</cited_answer>

Here are the documents:{context}"""

xml_prompt = ChatPromptTemplate.from_messages(
    [("system", xml_system), ("human", "{question}")]
)

def format_docs_xml(docs: List[Document]) -> str:
    formatted = []
    for i, doc in enumerate(docs):
        source_info = f"{doc.metadata.get('title', 'Unknown')}, page {doc.metadata.get('page', 'N/A')}, {doc.metadata.get('creationdate', 'N/A')[:4]}"
        doc_str = f"""\
        <source id=\"{i}\">
            <source_info>{source_info}</source_info>
            <content>{doc.page_content}</content>
        </source>"""
        formatted.append(doc_str)
    return "\n\n<sources>" + "\n".join(formatted) + "</sources>"


class State(TypedDict):
    question: str
    context: List[Document]
    citations: List[str]
    answer: Dict


def generate(state: State):
    formatted_docs = format_docs_xml(state["context"])
    messages = xml_prompt.invoke(
        {"question": state["question"], "context": formatted_docs}
    )
    llm = ChatOllama(model="gemma3:4b")
    response = llm.invoke(messages)
    parsed_response = XMLOutputParser().invoke(response.content)
    return {"answer": parsed_response, "citations": formatted_docs}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
import json

result = graph.invoke({"question": "Embodied Agent가 뭐야?"}, config = {"configurable": {"thread_id": "1"}})
print(json.dumps(result["answer"], indent=2))

In [ ]:
for citation in result["answer"]['cited_answer'][1]['citations']:
    print(f"📚 참고 문서: {citation['citation'][0]['source']}")
    print(f"💬 참고한 부분: {citation['citation'][1]['quote']}")

In [ ]:
print(result["answer"]['cited_answer'][0]['answer'])